This script evaluates the performance of different graph-aware architectures in a node classification problem. Several datasets are employed paying special attention to the homophily ratio.

In [6]:
import time
import numpy as np
from pandas import DataFrame
import matplotlib.pyplot as plt
import dgl
import networkx as nx
import torch
import torch.nn as nn


import utils
from gsp_utils.baselines_archs import GCNN_2L
from gsp_utils.baselines_models import NodeClassModel, GF_NodeClassModel
from gsp_utils.data import normalize_gso
from src.arch import GFGCN, GFGCNLayer, GFGCN_noh_Layer, GFGCN_Spows

# SEED = 0
SEED = 15
PATH = 'results/diff_filters/'
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
print(device)

torch.manual_seed(SEED)

cuda:1


In [7]:
def print_full_results(accs, ellapsed_time, datasets, exps):
    mean_accs = accs.mean(axis=2)
    med_accs = np.median(accs, axis=2)
    std_accs = accs.std(axis=2)
    mean_t = ellapsed_time.mean(axis=2)

    for i, dataset_name in enumerate(datasets):
        graph = getattr(dgl.data, dataset_name)(verbose=False)[0]
        edge_hom = dgl.edge_homophily(graph, graph.ndata['label'])

        print(f'{dataset_name} (homophily ratio: {edge_hom:.3f})')
        for j, exp in enumerate(exps):
            print(f'\t- {exp["leg"]}:\tmean: {mean_accs[j,i]:.3f} - std: {std_accs[j,i]:.4f} - med: {med_accs[j,i]:.3f} - time: {mean_t[j,i]:.2f} mins')
        
        print()

def summary_table(accs, datasets, exps, median=False):
    mean_accs = accs.mean(axis=2)
    cols_name = []
    for dataset_name in datasets:
        graph = getattr(dgl.data, dataset_name)(verbose=False)[0]
        edge_hom = dgl.edge_homophily(graph, graph.ndata['label'])
        cols_name.append(f'{dataset_name} ({edge_hom:.2f})')

    index_name = [exp['leg'] for exp in exps]

    return DataFrame(mean_accs, columns=cols_name, index=index_name)


In [8]:
DATASETS = ['TexasDataset',  'WisconsinDataset', 'CornellDataset', 'CiteseerGraphDataset', 'CoraGraphDataset']

EXPS = [{'model': 'Kipf', 'norm': 'both', 'leg': 'Kipf-normA'},
        {'model': 'Kipf', 'norm': 'none', 'leg': 'Kipf-A'},
        {'model': 'GFGCN', 'L': 2, 'K': 2, 'h0': .01, 'norm': True, 'leg': 'GCN-normA'},
        {'model': 'GFGCN', 'L': 2, 'K': 2, 'h0': .01, 'norm': False, 'leg': 'GCN-A'},
        {'model': 'GFGCN', 'L': 2, 'K': 2, 'h0': 1, 'norm': True, 'leg': 'GCN-normA-1'},
        {'model': 'GFGCN', 'L': 2, 'K': 2, 'h0': 1, 'norm': False, 'leg': 'GCN-A-1'},
        {'model': 'GFGCN', 'L': 4, 'K': 2, 'h0': .01, 'norm': False, 'leg': 'GCN-A-4-2'},
        {'model': 'H-GFGCN', 'L': 2, 'K': 2, 'norm': True, 'leg': 'GCN-normH'},
        {'model': 'H-GFGCN', 'L': 2, 'K': 2, 'norm': False, 'leg': 'GCN-H'},
        {'model': 'noh-GFGCN', 'L': 2, 'K': 2, 'norm': True, 'leg': 'W-GCN-normA'},
        {'model': 'noh-GFGCN', 'L': 2, 'K': 2, 'norm': False, 'leg': 'W-GCN-A'},]

## Original Parameters

In [4]:
# Common parameters - training
N_RUNS = 25
N_EPOCHS = 200
LR = .01
WD = 5e-4

# Common parameters - GNN
N_LAYERS = 2
K = 2
# IN_DIM = feat.shape[1]
# OUT_DIM = n_class
HID_DIM = 16
DROPOUT = 0
ACT = nn.ReLU()
LAST_ACT = nn.LogSoftmax(dim=1)
LOSS_FN = nn.NLLLoss()

In [5]:
best_accs = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
accs_best_val = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
ellapsed_times = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
for j, dataset in enumerate(DATASETS):
    A, feat, labels, n_class, masks = utils.get_data_dgl(dataset, dev=device)
    N = A.shape[0]
    in_dim = feat.shape[1]
    out_dim = n_class

    print(dataset)

    for i in range(N_RUNS):
        print(f'{i}:', end=' ')
        for k, exp in enumerate(EXPS):
            t_i = time.time()
            if exp['model'] == 'Kipf':
                arch = GCNN_2L(in_dim, HID_DIM, out_dim, act=ACT, last_act=LAST_ACT,
                               dropout=DROPOUT, norm=exp['norm'])
                S = dgl.from_networkx(nx.from_numpy_array(A)).add_self_loop().to(device)

            elif exp['model'] == 'GFGCN':
                arch = GFGCN(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                         dropout=DROPOUT, diff_layer=GFGCNLayer, init_h0=exp['h0'])
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)

            elif exp['model'] == 'H-GFGCN':
                arch = GFGCN_Spows(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                                   dropout=DROPOUT, norm=exp['norm'], dev=device)
                S = torch.Tensor(A).to(device)

            elif exp['model'] == 'noh-GFGCN':
                arch = GFGCN(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                             dropout=DROPOUT, diff_layer=GFGCN_noh_Layer)
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)
            else:
                raise Exception(f'ERROR: unknown architecture {exp["model"]}')

            if exp['model'] in ['Kipf', 'noh-GFGCN']:
                model = NodeClassModel(arch, S, masks, LOSS_FN, device=device)
            else:
                model = GF_NodeClassModel(arch, S, exp['K'], masks, LOSS_FN, device=device)

            loss, acc = model.train(feat, labels, N_EPOCHS, LR, WD)
            ellapsed_t = (time.time()-t_i)/60
            best_accs[k,j,i] = np.max(acc["test"])
            accs_best_val[k,j,i] = acc["test"][np.argmax(acc["val"])]

            ellapsed_times[k,j,i] = ellapsed_t

        print()      
    print()


print_full_results(accs_best_val, ellapsed_times, DATASETS, EXPS)
table_acc1 = summary_table(best_accs, DATASETS, EXPS)
table_acc_val1 = summary_table(accs_best_val, DATASETS, EXPS)

Extracting file to /home/srey/.dgl/texas_205d1148


FileNotFoundError: Cannot find DGL C++ sparse library at /home/srey/.local/lib/python3.8/site-packages/dgl/dgl_sparse/libdgl_sparse_pytorch_2.0.1.so

In [ ]:
table_acc_val1

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),CiteseerGraphDataset (0.74),CoraGraphDataset (0.81)
Kipf-normA,0.665946,0.592157,0.539459,0.71060,0.81392
Kipf-A,0.717838,0.578824,0.446486,0.65436,0.76372
GCN-normA,0.108108,0.019608,0.270270,0.69856,0.80584
GCN-A,0.696216,0.559216,0.574054,0.62272,0.72216
GCN-normA-1,0.108108,0.019608,0.270270,0.68836,0.77620
GCN-A-1,0.548108,0.477647,0.435676,0.63680,0.71756
GCN-A-4-2,0.654054,0.487843,0.580541,0.44868,0.56544
GCN-normH,0.610811,0.587451,0.499459,0.68404,0.77516
GCN-H,0.548108,0.476078,0.450811,0.63132,0.72000
W-GCN-normA,0.108108,0.019608,0.270270,0.69704,0.78272


In [ ]:
table_acc1

## Best Params GF-GNN

In [9]:
# BEST PARAMETERS
## Reaining params
N_RUNS = 5  # 25
N_EPOCHS = 200  # 500
LR = .05
WD = .01
DROPOUT = 0

# BEST PARAMETERS
## Architecture params
N_LAYERS = 2
K = 3
HID_DIM = 50

## Model params
h0 = 1
ACT = nn.ReLU()
LAST_ACT = nn.Softmax(dim=1)
LOSS_FN = nn.CrossEntropyLoss()

EXPS = [{'model': 'Kipf', 'norm': 'both', 'leg': 'Kipf-normA'},
        {'model': 'Kipf', 'norm': 'none', 'leg': 'Kipf-A'},
        {'model': 'GFGCN', 'L': N_LAYERS, 'K': K, 'h0': h0, 'norm': True, 'leg': 'A-GCN-normA'},
        {'model': 'GFGCN', 'L': N_LAYERS, 'K': K, 'h0': h0, 'norm': False, 'leg': 'A-GCN'},
        {'model': 'H-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': True, 'leg': 'H-GCN-normH'},
        {'model': 'H-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': False, 'leg': 'GCN-H'},
        {'model': 'noh-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': True, 'leg': 'W-GCN-normA'},
        {'model': 'noh-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': False, 'leg': 'W-GCN'},]

In [10]:
best_accs = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
accs_best_val = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
ellapsed_times = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
for j, dataset in enumerate(DATASETS):
    A, feat, labels, n_class, masks = utils.get_data_dgl(dataset, dev=device)
    N = A.shape[0]
    in_dim = feat.shape[1]
    out_dim = n_class

    print(dataset)

    for i in range(N_RUNS):
        print(f'{i}:', end=' ')
        for k, exp in enumerate(EXPS):
            t_i = time.time()
            if exp['model'] == 'Kipf':
                arch = GCNN_2L(in_dim, HID_DIM, out_dim, act=ACT, last_act=LAST_ACT,
                               dropout=DROPOUT, norm=exp['norm'])
                S = dgl.from_networkx(nx.from_numpy_array(A)).add_self_loop().to(device)

            elif exp['model'] == 'GFGCN':
                arch = GFGCN(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                         dropout=DROPOUT, diff_layer=GFGCNLayer, init_h0=exp['h0'])
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)

            elif exp['model'] == 'H-GFGCN':
                arch = GFGCN_Spows(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                                   dropout=DROPOUT, norm=exp['norm'], dev=device)
                S = torch.Tensor(A).to(device)

            elif exp['model'] == 'noh-GFGCN':
                arch = GFGCN(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                             dropout=DROPOUT, diff_layer=GFGCN_noh_Layer)
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)
            else:
                raise Exception(f'ERROR: unknown architecture {exp["model"]}')

            if exp['model'] in ['Kipf', 'noh-GFGCN']:
                model = NodeClassModel(arch, S, masks, LOSS_FN, device=device)
            else:
                model = GF_NodeClassModel(arch, S, exp['K'], masks, LOSS_FN, device=device)

            loss, acc = model.train(feat, labels, N_EPOCHS, LR, WD)
            ellapsed_t = (time.time()-t_i)/60
            
            
            best_accs[k,j,i] = np.max(acc["test"])
            accs_best_val[k,j,i] = acc["test"][np.argmax(acc["val"])]

            ellapsed_times[k,j,i] = ellapsed_t

            print(f'{accs_best_val[k,j,i]:.3f} ({best_accs[k,j,i]:.3f})', end=' - ')
        print()      
    print()


print_full_results(accs_best_val, ellapsed_times, DATASETS, EXPS)
table_acc2 = summary_table(best_accs, DATASETS, EXPS)
table_acc_val2 = summary_table(accs_best_val, DATASETS, EXPS)

TexasDataset
0: 0.703 - 0.649 - 0.108 - 0.919 - 0.811 - 0.892 - 0.108 - 0.784 - 
1: 0.703 - 0.676 - 0.108 - 0.865 - 0.811 - 0.892 - 0.108 - 0.730 - 
2: 0.730 - 0.649 - 0.108 - 0.919 - 0.703 - 0.919 - 0.108 - 0.757 - 
3: 0.730 - 0.703 - 0.108 - 0.892 - 0.757 - 0.892 - 0.108 - 0.757 - 
4: 0.730 - 0.730 - 0.108 - 0.892 - 0.865 - 0.865 - 0.108 - 0.757 - 
5: 0.703 - 0.676 - 0.108 - 0.892 - 0.919 - 0.892 - 0.108 - 0.811 - 
6: 0.703 - 0.703 - 0.108 - 0.892 - 0.811 - 0.892 - 0.108 - 0.784 - 
7: 0.703 - 0.703 - 0.108 - 0.865 - 0.865 - 0.919 - 0.108 - 0.730 - 
8: 0.703 - 0.649 - 0.108 - 0.946 - 0.676 - 0.892 - 0.108 - 0.730 - 
9: 0.730 - 0.649 - 0.108 - 0.892 - 0.784 - 0.838 - 0.108 - 0.757 - 
10: 0.703 - 0.649 - 0.108 - 0.703 - 0.811 - 0.892 - 0.108 - 0.784 - 
11: 0.703 - 0.703 - 0.108 - 0.892 - 0.811 - 0.892 - 0.108 - 0.757 - 
12: 0.703 - 0.676 - 0.108 - 0.892 - 0.730 - 0.865 - 0.108 - 0.784 - 
13: 0.703 - 0.676 - 0.108 - 0.892 - 0.838 - 0.865 - 0.108 - 0.757 - 
14: 0.703 - 0.703 - 0.108 - 0.8

In [11]:
table_acc_val2

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),CiteseerGraphDataset (0.74),CoraGraphDataset (0.81)
Kipf-normA,0.705946,0.669020,0.622703,0.23912,0.31976
Kipf-A,0.674595,0.551373,0.430270,0.58636,0.76588
A-GCN-normA,0.108108,0.019608,0.270270,0.72432,0.81584
A-GCN,0.883243,0.856471,0.863784,0.54728,0.55160
H-GCN-normH,0.800000,0.716863,0.694054,0.23916,0.32232
GCN-H,0.880000,0.838431,0.864865,0.53152,0.54136
W-GCN-normA,0.108108,0.019608,0.270270,0.24784,0.31256
W-GCN,0.757838,0.699608,0.624865,0.51028,0.44872


In [1]:
table_acc2

NameError: name 'table3' is not defined